# get data from TIMS
as far as I understand, TIMS gives data only on County level, so for all-California data we need to scrape files separately


TODO: so it doesn't load bigger files (~2Mb), not sure why or how to fix it, I tried chromedriver and firefox geckodriver</br>
I gave up and downloaded missing files manually

In [ ]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from datetime import datetime
import time
import os

In [ ]:
today = datetime.now().strftime('%d/%m/%Y')
login_url = 'https://tims.berkeley.edu/login.php'
data_url = 'https://tims.berkeley.edu/tools/query/index.php'

In [ ]:
# enter your login and password
username = input("Enter username: ")
password = input("Enter password: ")

In [ ]:
# Set the path to your local web driver
driver_path = 'F:\progs\chromedriver\chromedriver.exe'
# driver_path = 'F:/progs/geckodriver/geckodriver.exe'
# firefox_binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'
download_path = r'F:\work\streets_for_all\hsla_data_analysis\school_speed_limit\downloads\\'

chrome_options = Options()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument("--enable-logging")
prefs = {
    'download.default_directory': download_path,
    'directory_upgrade': True
}
chrome_options.add_experimental_option("prefs", prefs)

# firefox_options = webdriver.FirefoxOptions()
# firefox_options.headless = True
# firefox_options.set_preference("browser.download.folderList", 2)
# firefox_options.set_preference("browser.download.dir", download_path)
# firefox_options.set_preference("browser.download.useDownloadDir", True)
# #firefox_options.set_preference("browser.download.viewableInternally.enabledTypes", "")
# #firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream,application/vnd.ms-excel,text/csv")
# firefox_options.binary_location = firefox_binary_path

In [ ]:
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
# driver = webdriver.Firefox(executable_path=driver_path, options=firefox_options)

In [ ]:
# login to TIMS
driver.get(login_url)
username_field = driver.find_element(By.ID, 'e')
password_field = driver.find_element(By.ID, 'p')
username_field.send_keys(username)
password_field.send_keys(password)
driver.find_element_by_xpath('//button[@class="btn btn-success"]').click()
time.sleep(5)

In [ ]:
values = []
for i in range(1, 59):
    if len(str(i))==1:
        values.append('0'+str(i))
    else:
        values.append(str(i))

In [ ]:
for v in values:
    print(v)
    # open SWITRS Query
    driver.get(data_url)

    # enter date
    datefrom_field = driver.find_element(By.ID, 'startDate')
    dateto_field = driver.find_element(By.ID, 'endDate')
    datefrom_field.clear()
    datefrom_field.send_keys('01/01/2021')
    dateto_field.clear()
    dateto_field.send_keys('12/31/2021')

    # select county by value 01-58 and city "all"
    select_county = Select(driver.find_element_by_id('county'))
    select_county.select_by_value(v)
    select_city = Select(driver.find_element_by_id('city'))
    select_city.select_by_value('All')

    driver.find_element(
        By.XPATH,'//button[@class="btn btn-success pull-right"]'
    ).click()
    driver.find_element(By.ID, 'downloadButton').click()
    driver.find_element(By.ID, 'downloadResult').click()
    time.sleep(60)

    if os.path.exists(download_path+'Crashes.csv'):
        print('renaming file')
        last_file = [
            file for file in os.listdir(download_path)
            if file.endswith('.csv')&file.startswith('Crashes')
        ][0]
        print(last_file)
        os.rename(download_path+last_file, f'{download_path}download_{v}.csv')
    else: 
        print('no file')
        continue

In [ ]:
driver.quit()

In [ ]:
# combine all files into one
csv_files = [file for file in os.listdir(download_path) if file.endswith('.csv')]
combined_data = pd.DataFrame()
columns = ['CASE_ID',
           'COLLISION_DATE', 'COLLISION_TIME',
           'COUNTY', 'CITY',
           'NUMBER_KILLED', 'NUMBER_INJURED',
           'LONGITUDE', 'LATITUDE', 'POINT_X', 'POINT_Y'
          ]

for file in csv_files:
    file_path = os.path.join(download_path, file)
    data = pd.read_csv(file_path)
    combined_data = pd.concat([combined_data, data[columns]], ignore_index=True)

combined_data.to_csv('tims_data_california.csv', index=False)